In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from plotnine import *

In [27]:
def getdata(directory,type='csv',excludes=[]):
    df=None
    for filename in os.listdir(directory):
        if not filename.endswith('.zip') and filename not in excludes:
            if type=='csv':
                if df is None:
                    df=pd.read_csv(os.path.join(directory,filename))
                else:
                    df=df.merge(pd.read_csv(os.path.join(directory,filename)),left_on='ST_CASE',right_on='ST_CASE')
            else:
                if df is None:
                    df=pd.read_sas(os.path.join(directory,filename))
                else:
                    df=df.merge(pd.read_sas(os.path.join(directory,filename)),left_on='ST_CASE',right_on='ST_CASE')               
    return df

In [28]:
df15=getdata('data/2015')

MemoryError: 

In [ ]:
df15.head()